##### Source:
imblearn Documentation
- https://github.com/scikit-learn-contrib/imbalanced-learn 
- http://scikit-learn.org/stable/auto_examples/model_selection/plot_precision_recall.html

In [2]:
import zipfile
import pandas as pd
import numpy as np
import glob
from scipy import sparse
from sklearn.model_selection import train_test_split
from collections import Counter
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import KFold
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import confusion_matrix
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
# pubchem_zip = zipfile.ZipFile('PubChem.zip')
# pubchem_zip.extractall('PubChem_Molecule_Descriptor')
# pubchem_zip.close()

In [3]:
molecule_descriptor_files = []
for file in glob.glob('./Dataset/PubChem_Molecule_Descriptor/PubChem/AID_2289_descriptors_part[0-9]*.csv'):
    molecule_descriptor_files.append(file)

### Drug Bank

In [8]:
drug_link = pd.read_csv('./Dataset/drug_links.csv',dtype={'PubChem Compound ID':'str'})

In [9]:
drug_link = drug_link.dropna(subset=['PubChem Compound ID'])

In [10]:
drug_map = dict(zip(drug_link['PubChem Compound ID'],drug_link['Name']))

### Prepare training data

In [11]:
%%time
molecule_matrix = pd.read_csv(molecule_descriptor_files[0])
features = molecule_matrix.columns[1:180] # feature names except the first column (molecule ID)
molecule_matrix = molecule_matrix.iloc[:,1:180]
molecule_matrix = np.array(molecule_matrix.dropna(axis=0))

for file in molecule_descriptor_files[1:]:
    f = pd.read_csv(file)
    f = f.iloc[:,1:180] # the first column is molecule ID column
    f = np.array(f.dropna(axis=0)) # drop rows with NA 
    molecule_matrix = np.vstack((molecule_matrix,f))

CPU times: user 6.92 s, sys: 1.21 s, total: 8.13 s
Wall time: 8.79 s


In [12]:
molecule_matrix.shape

(307788, 179)

### label

In [12]:
ylabel = []
for file in molecule_descriptor_files:
    f = pd.read_csv(file)
    f.columns = map(str.lower,f.columns)
    for i in range(len(f['outcome'].dropna(axis=0))):
        ylabel.append(f['outcome'][i])
ylabel = np.array(ylabel)

In [13]:
ylabel.shape

(307788,)

In [14]:
ylabel[np.where(ylabel=='active')] = 1
ylabel[np.where(ylabel=='inactive')] = 0
ylabel = ylabel.astype(int)

In [15]:
Counter(ylabel)

Counter({0: 304501, 1: 3287})

### moleclue ID

In [16]:
# Prepare for molecule ID
moleculeID = []
for file in molecule_descriptor_files:
    f = pd.read_csv(file)
    for i in range(len(f['Row'].dropna(axis=0))):
        moleculeID.append(f['Row'][i])
moleculeID = np.array(moleculeID)

In [17]:
print(moleculeID.shape)

(307788,)


### Handling the imbalance

##### 1). Under-sampling: Random majority under-sampling with replacement

In [62]:
%%time
from imblearn.under_sampling import RandomUnderSampler

molecule_matrix = sparse.csr_matrix(molecule_matrix)

ratio = {0:3287, 1:3287}
undersam = RandomUnderSampler(ratio=ratio,return_indices=True)
X_resample,y_resample, idx_resample = undersam.fit_sample(molecule_matrix,ylabel)

# # under-sampling
# # make use of k-means
# # source: http://contrib.scikit-learn.org/imbalanced-learn/stable/under_sampling.html
# from imblearn.under_sampling import ClusterCentroids
# cc = ClusterCentroids(random_state=0)
# X_resample, y_resample = cc.fit_sample(xtrain,y_train)
# print(sorted(Counter(y_resample).items()))

CPU times: user 127 ms, sys: 12 ms, total: 139 ms
Wall time: 137 ms


### Split the train and test

In [63]:
X_resample.shape

(6574, 179)

In [64]:
x_train,x_test,y_train,y_test = train_test_split(X_resample,y_resample)

In [65]:
# active class ratio
y_tr = dict(Counter(y_train))
print(y_tr[1]/(y_tr[0]+y_tr[1]))
y_te = dict(Counter(y_test))
print(y_te[1]/(y_te[0]+y_te[1]))

0.5010141987829615
0.49695863746958635


### Random Forest Classifier

In [66]:
highest_cross_val_accu = -1
best_t = None
indices = range(x_train.shape[0])
tree = np.arange(10,500,50)
kf = KFold(n_splits=5,shuffle=True,random_state=28584096)

for n_tree in tree:
    accuracies = []
    for train_indices, val_indices in kf.split(indices):
        x_tr = x_train[train_indices]
        y_tr = y_train[train_indices]
        cf = RandomForestClassifier(n_estimators=n_tree)
        cf.fit(x_tr,y_tr)
        pred = cf.predict(x_train[val_indices])
        acc = accuracy_score(y_true=y_train[val_indices],y_pred=pred)
        accuracies.append(acc)
    cross_val_acc = np.mean(accuracies)
    print('N_tree: ', n_tree, 'cross validation accuracy:', cross_val_acc)
    if cross_val_acc > highest_cross_val_accu:
        highest_cross_val_accu = cross_val_acc
        best_t = n_tree
print('Best N_tree: ',best_t, '; cross-validation accuracy: ',highest_cross_val_accu)

N_tree:  10 cross validation accuracy: 0.52860040568
N_tree:  60 cross validation accuracy: 0.522718052738
N_tree:  110 cross validation accuracy: 0.530020283976
N_tree:  160 cross validation accuracy: 0.533671399594
N_tree:  210 cross validation accuracy: 0.530425963489
N_tree:  260 cross validation accuracy: 0.529006085193
N_tree:  310 cross validation accuracy: 0.534888438134
N_tree:  360 cross validation accuracy: 0.534482758621
N_tree:  410 cross validation accuracy: 0.528194726166
N_tree:  460 cross validation accuracy: 0.525963488844
Best N_tree:  310 ; cross-validation accuracy:  0.534888438134


In [67]:
rf_cv = RandomForestClassifier(n_estimators=best_t)
rf_cv.fit(x_train,y_train)
pred_cv_train = rf_cv.predict(x_train)
y_pred = rf_cv.predict(x_test)
print('Train accuracy:', accuracy_score(y_true=y_train,y_pred=pred_cv_train))
print('Test accuracy:', accuracy_score(y_true=y_test,y_pred=y_pred))

Train accuracy: 0.999188640974
Test accuracy: 0.53102189781


In [72]:
tn,fp,fn,tp = confusion_matrix(y_pred=y_pred,y_true=y_test).ravel()

In [73]:
confusion_matrix(y_pred=y_pred,y_true=y_test)

array([[458, 369],
       [402, 415]])

In [74]:
print('Recall:', tp/(tp+fn))
print('Precision:', tp/(tp+fp))

Recall: 0.507955936353
Precision: 0.529336734694


In [75]:
features[np.argsort(rf_cv.feature_importances_)[::-1][:5]]

Index(['XLogP', 'MW', 'WBN_LP_H_0.25', 'PSA', 'WBN_GC_H_0.25'], dtype='object')

In [31]:
# %%bash 
# pwd # print working directory 

/home/yating


### SVM

In [46]:
highest_cross_val_accu = -1
best_c = None
indices = range(x_train.shape[0])
c = [1e-7,1e-5,1e-3,1,10,100]
kf = KFold(n_splits=5, shuffle=True, random_state=28584096)

for c_ in c:
    accuracies = []
    for train_indices, val_indices in kf.split(indices):
        x_tr = x_train[train_indices]
        y_tr = y_train[train_indices]
        svm = SVC(kernel='rbf',C=c_)
        svm.fit(x_tr,y_tr)
        pred = svm.predict(x_train[val_indices])
        acc = accuracy_score(y_pred=pred,y_true=y_train[val_indices])
        accuracies.append(acc)
    cross_val_acc = np.mean(accuracies)
    print('C:', c_, ' cross validation accuracy:', cross_val_acc)
    if cross_val_acc > highest_cross_val_accu:
        highest_cross_val_accu = cross_val_acc
        best_c = c_
print('Best c:', best_c, '; cross-validation accuracy:',highest_cross_val_accu)

C: 1e-07  cross validation accuracy: 0.67362962963
C: 1e-05  cross validation accuracy: 0.67362962963
C: 0.001  cross validation accuracy: 0.67362962963
C: 1  cross validation accuracy: 0.666518518519
C: 10  cross validation accuracy: 0.618074074074
C: 100  cross validation accuracy: 0.566074074074
Best c: 1e-07 ; cross-validation accuracy: 0.67362962963


### RF:  Drug Bank

In [47]:
drugbank = pd.read_csv('DrugBank_MV.csv',dtype={'Row':'str'})
drugid = np.array(drugbank.iloc[:,0].dropna(axis=0))
drugbank = drugbank.iloc[:,1:180]
drugbank = np.array(drugbank.dropna(axis=0))
print(drugbank.shape)

(8722, 179)


In [48]:
drug_pred = rf_cv.predict(drugbank)
print(Counter(drug_pred))
# predicted active molucule
drugid_rf = drugid[np.where(drug_pred==1)]

Counter({0: 8104, 1: 618})


### SVM:  Drug Bank

In [49]:
drug_pred_svm = svm.predict(drugbank)
print(Counter(drug_pred_svm))
drugid_svm = drugid[np.where(drug_pred_svm==1)]

Counter({0: 6264, 1: 2458})


In [57]:
drug_name_rf = [drug_map[idx] for idx in drugid_rf]

In [58]:
drug_name_svm = [drug_map[idx] for idx in drugid_svm]

In [59]:
name = [set(drug_name_svm) & set(drug_name_rf)]